In [1]:
import pandas as pd
import numpy as np

#This file will complement the all_movies_cleanned, therefore I'll do some adjustments to avoid issues when I'll merge them.

In [2]:
a_oscars = pd.read_csv('the_oscar_award.csv')
display(a_oscars.head(60), a_oscars.shape)

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False
5,1927,1928,1,ART DIRECTION,Rochus Gliese,Sunrise,False
6,1927,1928,1,ART DIRECTION,William Cameron Menzies,The Dove;,True
7,1927,1928,1,ART DIRECTION,Harry Oliver,7th Heaven,False
8,1927,1928,1,CINEMATOGRAPHY,George Barnes,The Devil Dancer;,False
9,1927,1928,1,CINEMATOGRAPHY,Charles Rosher,Sunrise,True


(10765, 7)

# Initial data exploration:

In [3]:
# Check the first few rows of the dataset
a_oscars.head()

,year_film,year_ceremony,ceremony,category,name,film,winner
0,1927,1928,1,ACTOR,Richard Barthelmess,The Noose,False
1,1927,1928,1,ACTOR,Emil Jannings,The Last Command,True
2,1927,1928,1,ACTRESS,Louise Dresser,A Ship Comes In,False
3,1927,1928,1,ACTRESS,Janet Gaynor,7th Heaven,True
4,1927,1928,1,ACTRESS,Gloria Swanson,Sadie Thompson,False


In [4]:
# Check the number of rows and columns
a_oscars.shape


(10765, 7)

In [5]:
# Check the data types of each column
a_oscars.dtypes

year_film         int64
year_ceremony     int64
ceremony          int64
category         object
name             object
film             object
winner             bool
dtype: object

In [6]:
# Check for missing values in each column
a_oscars.isnull().sum()


year_film          0
year_ceremony      0
ceremony           0
category           0
name               4
film             315
winner             0
dtype: int64

In [7]:
# Check the number of unique values in each column
a_oscars.nunique()


year_film          95
year_ceremony      95
ceremony           95
category          115
name             6952
film             4991
winner              2
dtype: int64

In [8]:
# Check for duplicates in each column
for col in a_oscars.columns:
    print(f"{col}: {a_oscars.duplicated(col).sum()}")


year_film: 10670
year_ceremony: 10670
ceremony: 10670
category: 10650
name: 3812
film: 5773
winner: 10763


# After the first exploration I'm going to start cleanning

In [9]:
a_oscars.columns

Index(['year_film', 'year_ceremony', 'ceremony', 'category', 'name', 'film',
       'winner'],
      dtype='object')

In [10]:
# Use the rename method to change the column name to be the same of the all_films dataframe
a_oscars.rename(columns={'film': 'movie_name'}, inplace=True)


In [11]:
#In order to improve the dataset I'll use for the proect, I'm going to clean and append all awards categories together, dividing in nominate and won.
#Also it's important to eliminate duplicate movies

# Fill missing values in 'category' column with an empty string
a_oscars['category'].fillna('', inplace=True)

# Group by 'movie_name' and aggregate the 'category' values for rows with 'winner' as False, and concatenate them
# with comma separator to create 'osc_nominated_awards'
a_oscars['osc_nominated_awards'] = a_oscars[a_oscars['winner'] == False].groupby('movie_name')['category'].apply(', '.join)

# Convert 'osc_nominated_awards' values to strings
a_oscars['osc_nominated_awards'] = a_oscars['osc_nominated_awards'].astype(str)

# Group by 'movie_name' and aggregate the 'category' values for rows with 'winner' as True, and concatenate them
looser = a_oscars[a_oscars['winner'] == False].groupby('movie_name')['category'].apply(lambda x: ', '.join(x)).reset_index()

# Merge the winners dataframe with the original dataframe
a_oscars = pd.merge(a_oscars, looser, on='movie_name', how='left')

# Rename the concatenated category column
a_oscars.rename(columns={'category_x': 'category', 'category_y': 'osc_nominated_categories'}, inplace=True)

# Group by 'movie_name' and aggregate the 'category' values for rows with 'winner' as True, and concatenate them
# with comma separator to create 'osc_won_awards'
a_oscars['osc_won_awards'] = a_oscars[a_oscars['winner'] == True].groupby('movie_name')['category'].apply(', '.join)

# Convert 'osc_won_awards' values to strings
a_oscars['osc_won_awards'] = a_oscars['osc_won_awards'].astype(str)


# Group by 'movie_name' and aggregate the 'category' values for rows with 'winner' as True, and concatenate them
winner = a_oscars[a_oscars['winner'] == True].groupby('movie_name')['category'].apply(lambda x: ', '.join(x)).reset_index()

# Merge the winners dataframe with the original dataframe
a_oscars = pd.merge(a_oscars, winner, on='movie_name', how='left')

# Rename the concatenated category column
a_oscars.rename(columns={'category_x': 'category', 'category_y': 'osc_won_categories'}, inplace=True)

# Drop unnecessary columns
a_oscars.drop(['osc_won_awards','osc_nominated_awards','category','winner' ], axis=1, inplace=True)

# Drop the duplicate rows based on 'movie_name' and keep the last occurrence (which has 'winner' as True)
a_oscars = a_oscars.drop_duplicates(subset='movie_name', keep='last')



In [12]:
#verifiying new shape
a_oscars.shape

(4992, 7)

In [13]:
# I want to add the number of nominations or awards

# define a function to count number of categories through commas
def count_commas(s):
    if pd.notna(s):
        return len(s.split(','))
    else:
        return 0

# Apply the function to the columns 'osc_nominated_categories' and 'osc_won_categories'
a_oscars['osc_nominated_num'] = a_oscars['osc_nominated_categories'].apply(count_commas)
a_oscars['osc_won_num'] = a_oscars['osc_won_categories'].apply(count_commas)


In [14]:
#As there aren't adding valuable information I'll drop few columns
a_oscars.drop(['year_ceremony','ceremony','year_film' ], axis=1, inplace=True)


In [15]:
#In order to be self explainatory I rename name

a_oscars.rename(columns={'name': 'person_awarded'}, inplace=True)


In [16]:
a_oscars

,person_awarded,movie_name,osc_nominated_categories,osc_won_categories,osc_nominated_num,osc_won_num
0,Richard Barthelmess,The Noose,ACTOR,NaN,1,0
2,Louise Dresser,A Ship Comes In,ACTRESS,NaN,1,0
4,Gloria Swanson,Sadie Thompson,ACTRESS,NaN,1,0
6,William Cameron Menzies,The Dove;,NaN,ART DIRECTION,0,1
8,George Barnes,The Devil Dancer;,CINEMATOGRAPHY,NaN,1,0
...,...,...,...,...,...,...
10756,Written by Steven Spielberg & Tony Kushner,The Fabelmans,"ACTOR IN A SUPPORTING ROLE, ACTRESS IN A LEADI...",NaN,7,0
10757,Written by Todd Field,Tár,"ACTRESS IN A LEADING ROLE, CINEMATOGRAPHY, DIR...",NaN,6,0
10758,Written by Ruben Östlund,Triangle of Sadness,"DIRECTING, BEST PICTURE, WRITING (Original Scr...",NaN,3,0
10763,Iain Neil,NaN,NaN,NaN,0,0


In [17]:
# Remove the 'person_awarded' column
a_oscars = a_oscars.drop('person_awarded', axis=1)

# Split text in 'osc_nominated_categories' by ',' and create new columns for each category
osc_nominated_categories = a_oscars['osc_nominated_categories'].str.split(',', expand=True)
osc_nominated_categories.columns = [f'osc_nominated_category{i+1}' for i in range(osc_nominated_categories.shape[1])]
a_oscars = pd.concat([a_oscars, osc_nominated_categories], axis=1)
a_oscars = a_oscars.drop('osc_nominated_categories', axis=1)

# Split text in 'osc_won_categories' by ',' and create new columns for each category
osc_won_categories = a_oscars['osc_won_categories'].str.split(',', expand=True)
osc_won_categories.columns = [f'osc_won_category{i+1}' for i in range(osc_won_categories.shape[1])]
a_oscars = pd.concat([a_oscars, osc_won_categories], axis=1)
a_oscars = a_oscars.drop('osc_won_categories', axis=1)

# Show the resulting DataFrame
a_oscars

,movie_name,osc_nominated_num,osc_won_num,osc_nominated_category1,osc_nominated_category2,osc_nominated_category3,osc_nominated_category4,osc_nominated_category5,osc_nominated_category6,osc_nominated_category7,...,osc_won_category3,osc_won_category4,osc_won_category5,osc_won_category6,osc_won_category7,osc_won_category8,osc_won_category9,osc_won_category10,osc_won_category11,osc_won_category12
0,The Noose,1,0,ACTOR,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A Ship Comes In,1,0,ACTRESS,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sadie Thompson,1,0,ACTRESS,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,The Dove;,0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,The Devil Dancer;,1,0,CINEMATOGRAPHY,None,None,None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10756,The Fabelmans,7,0,ACTOR IN A SUPPORTING ROLE,ACTRESS IN A LEADING ROLE,DIRECTING,MUSIC (Original Score),BEST PICTURE,PRODUCTION DESIGN,WRITING (Original Screenplay),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10757,Tár,6,0,ACTRESS IN A LEADING ROLE,CINEMATOGRAPHY,DIRECTING,FILM EDITING,BEST PICTURE,WRITING (Original Screenplay),None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10758,Triangle of Sadness,3,0,DIRECTING,BEST PICTURE,WRITING (Original Screenplay),None,None,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10763,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
a_oscars.to_csv('a_oscars_cleaned.csv', index=False)
